In [ ]:
from dataclasses import dataclass
from typing import NamedTuple, Optional, Callable
from decimal import Decimal
from collections.abc import Sequence
from abc import ABC, abstractmethod

class Customer(NamedTuple):
    name: str
    loyalty: int

class Item(NamedTuple):
    product: str
    quantity: int
    price: Decimal

    def total(self) -> Decimal:
        return self.price * self.quantity

@dataclass(frozen=True)
class Order(NamedTuple):
    customer: Customer
    cart: Sequence[Item]
    promotion: Optional[Callable[['Order'], Decimal]] = None

    def total(self) -> Decimal:
        totals = (item.total() for item in self.cart)
        return sum(totals, start=Decimal(0))

    def due(self) -> Decimal:
        if self.promotion is None:
            discount = Decimal(0)
        else:
            discount = self.promotion(self)

        return self.total() - discount

    def __repr__(self):
        return f'<Order total: {self.total()():.2f} due: {self.due():.2f}>'


class Promotion(ABC): # abstract class
    @abstractmethod
    def discount(self, order: Order) -> Decimal:
        """
        Return discount
        """

class LoyaltyPromo(Promotion):

    def discount(self, order: Order) -> Decimal:
        rate = Decimal('0.05')
        if order.customer.loyalty >= 1000:
            return order.total() * rate
        return Decimal(0)